<a href="https://colab.research.google.com/github/SSAC-AI/resnet/blob/main/Bottleneck_Block.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

| 인자                        | 설명                                |
| ------------------------- | --------------------------------- |
| `in_channels`             | 입력 텐서의 채널 수                       |
| `out_channels`            | 출력 채널 수 (shortcut 포함 최종 출력)       |
| `bottleneck_channels`     | 중간 bottleneck 레이어의 채널 수 (작은 값)    |
| `stride`                  | 다운샘플링 여부 (1 또는 2)                 |
| `dilation`                | Atrous convolution을 위한 dilation 값 |
| `use_bounded_activations` | ReLU6와 clip 연산을 사용할지 여부 (양자화에 유리) |


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Bottleneck(nn.Module): # ResNet의 Bottleneck 블록을 나타냄
    def __init__(self, in_channels, out_channels, bottleneck_channels, stride=1,
                 dilation=1, use_bounded_activations=False):
        super(Bottleneck, self).__init__()

        self.use_bounded_activations = use_bounded_activations
        self.stride = stride

        # 1x1 conv: dimension reduction 채널 수 감소.
        self.conv1 = nn.Conv2d(in_channels, bottleneck_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(bottleneck_channels)

        # 3x3 conv: feature extraction 특성 추출
        self.conv2 = nn.Conv2d(bottleneck_channels, bottleneck_channels, kernel_size=3,
                               stride=stride, padding=dilation, dilation=dilation, bias=False) # stride가 2이면 downsampling 수행
        self.bn2 = nn.BatchNorm2d(bottleneck_channels)

        # 1x1 conv: dimension restore 채널 복원, 활성화 함수 없음
        self.conv3 = nn.Conv2d(bottleneck_channels, out_channels, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels)

        # Shortcut (identity or projection)
        if in_channels != out_channels or stride != 1:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            ) # 입력 채널 수와 출력 채널 수가 다르거나 stride가 2인 경우 1x1 Conv + BN으로 shape을 맞춰줌
        else:
            self.shortcut = nn.Identity() # 그렇지 않으면 그대로 shortcut 연결

    def forward(self, x):
        shortcut = self.shortcut(x) # 입력 x를 shortcut경로로 보냄

        # Conv -> BatchNorm -> ReLu6
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu6(out) if self.use_bounded_activations else F.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = F.relu6(out) if self.use_bounded_activations else F.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        out += shortcut
        if self.use_bounded_activations: # True면 clip_by_value와 유사한 torch.clamp로 -6~6 사이로 값 제한
            out = torch.clamp(out, -6.0, 6.0)
            out = F.relu6(out)
        else:
            out = F.relu(out)
        return out

torch.Size([1, 256, 56, 56])
